# 가변수화

## 1.환경준비

### (1) 라이브러리 불러오기

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#전처리를 위한 함수
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore')

### (2) 데이터 불러오기

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv'
data = pd.read_csv(path)
data.head()

|	변수명	|	설명	|	구분	|
|----|----|----|
|	Sales 	|	 각 지역 판매량(단위 : 1000개)	|	Target	|
|	CompPrice 	|	지역별 경쟁사 판매가격(달러)	|	feature	|
|	Income 	|	가구당 평균 소득액(1000달러)	|	feature	|
|	Advertising 	|	 각 지역, 회사의 광고 예산(1000달러)	|	feature	|
|	Population 	|	 지역 인구수(단위 : 1000명)	|	feature	|
|	Price 	|	 자사 지역별 판매가격(달러)	|	feature	|
|	ShelveLoc 	|	 진열상태(범주 : Bad, Medium, Good)	|	feature	|
|	Age 	|	 지역 인구의 평균 연령	|	feature	|
|	Education 	|	 교육수준(범주 : 10~18)	|	feature	|
|	Urban 	|	 매장이 도심에 있는지 여부(범주 : Yes, No)	|	feature	|
|	US 	|	 매장이 미국에 있는지 여부(범주 : Yes, No)	|	feature	|


## 2.가변수화

### (1) 예제 데이터

* 샘플데이터 생성

In [ ]:
train = pd.DataFrame({'x1':['a','b','a','b','c'], 'x2':[2,6,5,3,4]})
test = pd.DataFrame({'x1':['a','b','b'], 'x2':[7,8,9]})

In [ ]:
train

In [ ]:
test

* 가변수화

In [ ]:
pd.get_dummies(train, columns = ['x1'])

In [ ]:
pd.get_dummies(test, columns = ['x1'])

* 어떻게 해결해야 할까요?

### (2) pd.Categorical
* help : https://pandas.pydata.org/docs/reference/api/pandas.Categorical.html

In [ ]:
train['x1'] = pd.Categorical(train['x1'], categories = ['a','b','c'])
train['x1']

In [ ]:
test['x1'] = pd.Categorical(test['x1'], categories = ['a','b','c'])

In [ ]:
pd.get_dummies(train, columns = ['x1'])

In [ ]:
pd.get_dummies(test, columns = ['x1'])

## 3.실습
* (1) test 데이터를 5건 분할합니다.
* (2) 나머지를 가지고
    * 1) x, y 분할
    * 2) 범주형 변수에 대한 카테고리들을 정의하고, pd.Categorical 함수로 변환합니다.
        * 범주형 feature : ShelveLoc, Education, US, Urban
    * 3) 가변수화를 수행합니다.
    * 4) train, val로 분할합니다.
* (3) 모델링 
    * 간단한 회귀모델을 생성하고 검증합니다.
* (4) test를 가지고
    * 1) y를 떼어내고
    * 2) 가변수화 : 2번에서 정의한 카테고리로 pd.Categorical 함수 실행 후 가변수화 수행
    * 3) 모델로 예측합니다. 

### (1) test 분리

In [ ]:
train, test = train_test_split(data, test_size = 5, random_state = 10)

In [ ]:
test

### (2) 전처리

* x, y 분할

In [ ]:
target = 'Sales'
x = train.drop(target, axis = 1)
y = train.loc[:, target]

* Categorical로 변환하기

In [ ]:
# ShelveLoc, Urban , US
print(x['ShelveLoc'].unique())
print(x['Urban'].unique())
print(x['US'].unique())

In [ ]:
cat_cols = { 'ShelveLoc':['Bad','Medium','Good'],
             'Urban':['No', 'Yes'],
             'US' : ['No', 'Yes']}

for var, values in cat_cols.items() :
    x[var] = pd.Categorical(x[var], categories= values)

In [ ]:
x.info()

* 가변수화

In [ ]:
x = pd.get_dummies(x, columns = cat_cols.keys(), drop_first = True)

In [ ]:
x.head()

* train, val 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = .2)

### (3) 모델링

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import *

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
pred = model.predict(x_val)

print(mean_absolute_error(y_val, pred))

### (4) test 처리

* pd.Categorical을 이용하여 범주형 타입으로 변환하고, 가변수화 하기
* 모델 가지고 예측하기

In [ ]:
x_test = test.drop(target, axis = 1)

In [ ]:
for var, values in cat_cols.items() :
    x_test[var] = pd.Categorical(x_test[var], categories= values)

x_test = pd.get_dummies(x_test, columns = cat_cols.keys(), drop_first = True)

In [ ]:
model.predict(x_test)

In [ ]:
cat_cols